# Packages, Libararies, Loading Dataset and Functions

In [ ]:
# Import the necessary modules
import tensorflow as tf
import keras
import pickle
from sklearn.model_selection import train_test_split
import numpy as np 
import math
import matplotlib.pyplot as plt
from numpy import loadtxt
import os
from google.colab import drive


In [ ]:
# Connect to Google Drive
drive.mount('/content/drive')

# Direct to the correct directory
os.chdir("drive/My Drive/SEPSIS")

# Check the files
!ls


In [ ]:
# Load the dataset including the labels

with open("X_train.txt", "rb") as fp:   # Unpickling
  X = pickle.load(fp)

with open("y_train.txt", "rb") as fp:   # Unpickling
  y = pickle.load(fp)

# Split the x (data) and Y (labels) sets into train and test splits

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1111111, random_state=42)


In [ ]:
# The loss function is used to determine the error between the output (y_pred) and the target value (y_true)
# The Cross Entropy Loss function is modified by adding weights to "no sepsis" label, then the loss function is returned

def keras_custom_loss_function(y_true, y_pred):
  custom_loss = -y_true*tf.math.log(y_pred) - (1-y_true)*tf.math.log(1-y_pred) * 0.7
  return custom_loss

# This function takes no argument, it returns the archticture of the model 
# Which is compiled using the tf.keras.layers and also
# including the optimzer, learning rate, loss function and evaluation metrics


def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0., input_shape=(336,40)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GRU(400, dropout=0.2, recurrent_dropout=0.5, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GRU(400, dropout=0.2, recurrent_dropout=0.5, return_sequences=True),   
    tf.keras.layers.BatchNormalization(),     
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.BatchNormalization(),      
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.BatchNormalization(),      
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),      
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.BatchNormalization(),      
    tf.keras.layers.Dense(2,  activation='softmax'),   
  ])

  opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
  
  model.compile(optimizer=opt,
                loss=keras_custom_loss_function,
                metrics=[tf.keras.metrics.AUC()])
  return model


In [ ]:
# This function takes a number as input, to specify the model that is made
# It uses the get_compiled() to set the model's architecture and parameters
# such as the optimizer, loss function, learning rate, epochs, 
# This model is then trained using the X_train and y_train and validated using X_test and y_test
# After training the model's architecture, weights and training configurations are saved in a .h5 file

def create_model(number_of_model):

  print("_______________________________________________________\n")
  print("\033[1mCompiling and Training #{}_GRU_Model\033[0m".format(number_of_model))
  print("_______________________________________________________\n")
  model = get_compiled_model()
  model.summary()
  original = model.fit(X_train,y_train, validation_data=(X_test, y_test), batch_size=64, epochs=10)

  print("\033[1mFinished Compiling and Training model\033[0m")
  
  print("\n\033[1mSaving #{}_GRU_Model\033[0m\n".format(number_of_model))
  model.save("#{}_GRU_Model.h5".format(number_of_model))
  print("\n \033[1m#{}_GRU_Model saved to disk\033[0m\n".format(number_of_model))


# Models


As six GRU models are required for the ensemble of GRU models, first six model are compiled and trained (using the create_model()and get_compiled_model() functions) and each is saved using the model.save() function which saves the model's architecture, weights, and training configuration in a single file.

In [ ]:
# The first GRU model
create_model(1)

In [ ]:
# The second GRU model
create_model(2)

In [ ]:
# The third GRU model
create_model(3)

In [ ]:
# The fourth GRU model
create_model(4)

In [ ]:
# The fifth GRU model
create_model(5)

In [ ]:
# The sixth GRU model
create_model(6)